In [123]:
%run ./a/import.ipynb
%run ./a/aroon.ipynb
%run ./a/bollinger_bands.ipynb
%run ./a/CHOP.ipynb
%run ./a/DMI.ipynb
%run ./a/EMA_MACD.ipynb
%run ./a/MFI.ipynb
%run ./a/OBV.ipynb
%run ./a/OHLC.ipynb
%run ./a/pivot_pt_hl.ipynb
%run ./a/pivot_pt_rs.ipynb
%run ./a/ROC.ipynb
%run ./a/RSI.ipynb
%run ./a/close_stochastic.ipynb

In [124]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.core.display import display, HTML
%matplotlib notebook

In [125]:
company = 'MSFT'
API = 'yahoo'
start = '2016-01-01'
end = '2019-12-31'

In [126]:
df = data.DataReader(company, API, start, end)

In [129]:
def plot(Aroon=False, Bollinger_Bands=False, CHOP=False, DMI=False, MACD=False, MFI=False, OBV=False, OHLC=False, OHLC_day=10, 
         Pivot_HL=False, HL_period=21, Pivot_RS=False, RS_period=21, ROC=False, RSI=False, Close_Stochastic=False):
    global df
    df_ohlc = None
    df_pivot_rs = None
    
    if Aroon:
        if not 'aroon_up' in df:
            df1 = aroon(df)
            df = pd.concat([df, df1], axis=1)

        fig, (ax1,ax2) = plt.subplots(nrows=2, ncols=1,figsize=(12.2,9),sharex=True)

        df[['aroon_up','aroon_down']].plot(ax=axes[0]); axes[0].set_title('Aroon')
        df['aroon_diff'].plot(ax=axes[1], label='aroon_diff'); axes[1]
        
    if Bollinger_Bands:
        if not 'BOLU' in df:
            df1 = bollinger(df)
            df = pd.concat([df, df1], axis=1)
        
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12.2, 13.5), sharex=True)

        ax1.set_title('Bollinger Bands')
        ax1.plot(df.index, df['SMA'], color='orange')
        ax1.plot(df.index, df['BOLU'], color='red')
        ax1.plot(df.index, df['BOLD'], color='green')
        ax1.plot(df.index, df['Close'], color='blue')
        ax1.fill_between(df.index, df['BOLU'],
                         df['BOLD'], alpha=0.2, color='grey')

        ax2.plot(df.index, df['%b'], label='%b')

        ax3.plot(df.index, df['band_width'], label='band_width')
        
        ax2.legend()
        ax3.legend()
    
    if CHOP:
        if not 'ATR' in df:
            df1 = chop(df)
            df = pd.concat([df, df1], axis=1)
            
        fig, ax1 = plt.subplots(1, 1, figsize=(12.2,4.5))
        
        ax1.set_title('CHOP')
        ax1.plot(df.index, df.filter(regex='^CHOP_', axis=1))
        ax1.axhline(61.8, linestyle='--', alpha = 0.5, color = 'red', label='Over Bought')
        ax1.axhline(38.2, linestyle='--', alpha = 0.5, color = 'green', label='Over Sold')
        plt.legend(loc='lower right')
        
    if DMI:
        if not 'smooth_+DI' in df:
            df1 = dmi(df)
            df = pd.concat([df, df1], axis=1)
            
        fig, axes = plt.subplots(nrows=2, ncols=1,figsize=(12.2,9),sharex=True)

        df[['smooth_+DMI', 'smooth_-DMI']].plot(ax=axes[0]); axes[0].set_title('DMI')
        df[['ADXR', 'ADX']].plot(ax=axes[1]); axes[1].set_title('ADX'); axes[1].axhline(25, linestyle='--', alpha = 0.5, color = 'red')

    if MACD:
        if not 'MACD' in df:
            df1 = macd(df)
            df = pd.concat([df, df1], axis=1)
        
        fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12.2,9))

        ax1.plot(df.index, df.filter(regex='ema$', axis=1).iloc[:,0], 
                 label=[col for col in df.columns if 'ema' in col][0])
        ax1.plot(df.index, df.filter(regex='ema$', axis=1).iloc[:,1], 
                 label=[col for col in df.columns if 'ema' in col][1])
        ax1.legend()

        ax2.set_title('MACD')
        ax2.plot(df[['MACD', 'signal']])
        ax2.plot(df.index, df['MACD_up'], color='g')
        ax2.plot(df.index, df['MACD_down'], color='r')
        ax2.fill_between(df.index, df['MACD_up'], 0, alpha=0.3, color='green')
        ax2.fill_between(df.index, df['MACD_down'], 0, alpha=0.3, color='red')

    if MFI:
        if not 'MFI' in df:
            df1 = mfi(df)
            df = pd.concat([df, df1], axis=1)
            
        fig, ax = plt.subplots(1, 1, sharex=True, figsize=(12.2,4.5))
        
        ax.set_title('MFI')
        ax.plot(df['MFI'])
        
    if OBV:
        if not 'OBV' in df:
            df1 = obv(df)
            df = pd.concat([df, df1], axis=1)
            
        fig, ax1 = plt.subplots(1, 1, sharex=True, figsize=(12.2,4.5))
        
        ax1.set_title('OBV')
        ax1.plot(df.index, df['OBV'])

    if ROC:
        if not 'ROC' in df:
            df1 = roc(df)
            df = pd.concat([df, df1], axis=1)
        
        fig, ax1 = plt.subplots(1, 1, sharex=True, figsize=(12.2,4.5))
        
        ax1.set_title('ROC')
        ax1.plot(df.index, df['ROC'])
        
    if OHLC:
        if df_ohlc is None:
            df_ohlc = ohlc(df, OHLC_day)
        
        fig, ax1 = plt.subplots(1, 1, figsize=(12.2,4.5))
        
        candlestick_ohlc(ax1, df_ohlc.values, width=OHLC_day/1.8, colorup='g')

        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

    if Pivot_HL:
        if not 'h_index' in df:
            df1 = pivot_hl(df, HL_period)
            df = pd.concat([df, df1], axis=1)
        
        fig, ax = plt.subplots(1,1,figsize=(12.2,4.5))

        ax.plot(df.index, df['Close'])
        for i in range(len(df.index)):
            if df['h_index'][i]:
                ax.plot(df.index[i], df['High'][i], marker=7, color='red', label='Pivot High')
            elif df['l_index'][i]:
                ax.plot(df.index[i], df['Low'][i], marker=6, color='green', label='Pivot Low')

    if Pivot_RS:
        if df_pivot_rs is None:
            df_pivot_rs = pivot_rs(df, HL_period)
        
        OHLC = True
    
        fig, ax = plt.subplots(figsize=(12.2,4.5))
        
        for i in range( len(df_pivot_rs.index) - 1 ):
            plt.plot_date([df_pivot_rs.index[i], df_pivot_rs.index[i+1]], 
                         [df_pivot_rs['pivot_point'][i], df_pivot_rs['pivot_point'][i]], linestyle="-", lw=1, marker=",", color='blue', alpha = 0.5)
            plt.plot_date([df_pivot_rs.index[i], df_pivot_rs.index[i+1]], 
                         [df_pivot_rs['s1'][i], df_pivot_rs['s1'][i]], linestyle="--", lw=1, marker=",", color='red', alpha = 0.5)
            plt.plot_date([df_pivot_rs.index[i], df_pivot_rs.index[i+1]], 
                          [df_pivot_rs['s2'][i], df_pivot_rs['s2'][i]], linestyle="-.", lw=1, marker=",", color='orange', alpha = 0.5)
            plt.plot_date([df_pivot_rs.index[i], df_pivot_rs.index[i+1]], 
                         [df_pivot_rs['r1'][i], df_pivot_rs['r1'][i]], linestyle="--", lw=1, marker=",", color='red', alpha = 0.5)
            plt.plot_date([df_pivot_rs.index[i], df_pivot_rs.index[i+1]], 
                         [df_pivot_rs['r2'][i], df_pivot_rs['r2'][i]], linestyle="-.", lw=1, marker=",", color='orange', alpha = 0.5)
            plt.plot_date([df_pivot_rs.index[i], df_pivot_rs.index[i+1]], 
                         [df_pivot_rs['s3'][i], df_pivot_rs['s3'][i]], linestyle=":", lw=1, marker=",", color='yellow', alpha = 0.8)
            plt.plot_date([df_pivot_rs.index[i], df_pivot_rs.index[i+1]], 
                         [df_pivot_rs['r3'][i], df_pivot_rs['r3'][i]], linestyle=":", lw=1, marker=",", color='yellow', alpha = 0.8)

    if RSI:
        if not 'RSI' in df:
            df1 = rsi(df)
            df = pd.concat([df, df1], axis=1)

        plt.figure(figsize=(12.2,4.5))
        plt.title('RSI')
        plt.plot(df.index, df['RSI'])
        plt.axhline(0, linestyle='--', alpha = 0.5, color = 'red')
        plt.axhline(10, linestyle='--', alpha = 0.5, color = 'orange')
        plt.axhline(20, linestyle='--', alpha = 0.5, color = 'yellow')
        plt.axhline(30, linestyle='--', alpha = 0.5, color = 'grey')
        plt.axhline(70, linestyle='--', alpha = 0.5, color = 'grey')
        plt.axhline(80, linestyle='--', alpha = 0.5, color = 'yellow')
        plt.axhline(90, linestyle='--', alpha = 0.5, color = 'orange')
        plt.axhline(100, linestyle='--', alpha = 0.5, color = 'red')
    
    if Close_Stochastic:
        if not '%K-%D' in df:
            df1 = close_stochastic(df)
            df = pd.concat([df, df1], axis=1)

        fig, axes = plt.subplots(nrows=2, ncols=1,figsize=(12.2,9),sharex=True)

        df[['%K','%D']].plot(ax=axes[0]); axes[0].set_title('Close Oscillator')
        axes[0].axhline(20, linestyle='--', alpha = 0.5, color = 'green')
        axes[0].axhline(80, linestyle='--', alpha = 0.5, color = 'red')
        
        (df['%K'] - df['%D']).plot(ax=axes[1]); axes[1].set_title('%K-%D')
        axes[1].axhline(0, linestyle='--', alpha = 0.5, color = 'green')

interact_manual(plot, 
                a=widgets.Checkbox(Aroon=False, Bollinger_Bands=False, CHOP=False, DMI=False, MACD=False, MFI=False, OBV=False, OHLC=False), 
                OHLC_day=widgets.IntSlider(min=1, max=30, step=1,value=10), 
                b=widgets.Checkbox(Pivot_HL=False), 
                HL_period=widgets.IntSlider(min=1, max=99, step=2,value=21), 
                c=widgets.Checkbox(Pivot_RS=False), 
                RS_period=widgets.IntSlider(min=5, max=100, step=1,value=26), 
                d=widgets.Checkbox(ROC=False, RSI=False, Close_Stochastic=False))

interactive(children=(Checkbox(value=False, description='Aroon'), Checkbox(value=False, description='Bollinger…

<function __main__.plot(Aroon=False, Bollinger_Bands=False, CHOP=False, DMI=False, MACD=False, MFI=False, OBV=False, OHLC=False, OHLC_day=10, Pivot_HL=False, HL_period=21, Pivot_RS=False, RS_period=21, ROC=False, RSI=False, Close_Stochastic=False)>